<a href="https://colab.research.google.com/github/jianfeiZhao/M5-Sales-Forecasting/blob/main/proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the challenge, you are predicting item sales at stores in various locations for two 28-day time periods.

 **Files**

---


* calendar.csv - Contains information about the dates on which the products are sold.
* sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
* sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.
* sell_prices.csv - Contains information about the price of the products sold per store and date.
* sales_train_evaluation.csv - Includes sales [d_1 - d_1941] (**labels** used for the Public leaderboard)

This competition uses a **Weighted Root Mean Squared Scaled Error** (RMSSE)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Dataset

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Walmart Sales Pred/data/sales_train_validation.csv', encoding='unicode_escape')
dataset.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,4.0,1.0,0.0,1.0,3.0,7.0,2.0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,4.0


In [ ]:
print(dataset.shape)

(14914, 1919)


In [ ]:
print(dataset['cat_id'].value_counts(), '\n')
print(dataset['dept_id'].value_counts(), '\n')
print(dataset['state_id'].value_counts(), '\n')
print(dataset['store_id'].value_counts(), '\n')

FOODS        6853
HOUSEHOLD    5235
HOBBIES      2825
Name: cat_id, dtype: int64 

FOODS_3        3783
HOUSEHOLD_1    2660
HOUSEHOLD_2    2575
HOBBIES_1      2080
FOODS_2        1990
FOODS_1        1080
HOBBIES_2       745
Name: dept_id, dtype: int64 

CA    12196
TX     2717
Name: state_id, dtype: int64 

CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    2717
Name: store_id, dtype: int64 



In [ ]:
print(dataset['item_id'].value_counts(), '\n')

HOBBIES_1_001    5
FOODS_1_207      5
FOODS_1_199      5
FOODS_1_200      5
FOODS_1_201      5
                ..
FOODS_3_602      4
FOODS_3_601      4
FOODS_3_600      4
FOODS_3_599      4
FOODS_3_827      4
Name: item_id, Length: 3049, dtype: int64 



3049 different items 

14914 different id (storeId_itemId)

# Sell Prices

In [ ]:
prices = pd.read_csv('/content/drive/MyDrive/Walmart Sales Pred/data/sell_prices.csv', encoding='unicode_escape')
print(prices.head())
print(prices.shape)

  store_id        item_id  wm_yr_wk  sell_price
0     CA_1  HOBBIES_1_001   11325.0        9.58
1     CA_1  HOBBIES_1_001   11326.0        9.58
2     CA_1  HOBBIES_1_001   11327.0        8.26
3     CA_1  HOBBIES_1_001   11328.0        8.26
4     CA_1  HOBBIES_1_001   11329.0        8.26
(1970035, 4)


In [ ]:
prices['id'] = prices['item_id']+'_'+prices['store_id']
prices.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325.0,9.58,HOBBIES_1_001_CA_1
1,CA_1,HOBBIES_1_001,11326.0,9.58,HOBBIES_1_001_CA_1
2,CA_1,HOBBIES_1_001,11327.0,8.26,HOBBIES_1_001_CA_1
3,CA_1,HOBBIES_1_001,11328.0,8.26,HOBBIES_1_001_CA_1
4,CA_1,HOBBIES_1_001,11329.0,8.26,HOBBIES_1_001_CA_1


In [ ]:
s = prices['id'].value_counts()
print(s.value_counts(), '\n')

282    3352
281     265
280     142
57      128
276      78
       ... 
37        1
48        1
63        1
69        1
1         1
Name: id, Length: 250, dtype: int64 



dataset cover 282 weeks;

8883 sku have price, in which 3352 have complete price info